# PART 4: adding ipywidgets interaction to the ipycytoscape graph

### Objective
The objective of this article is learning how to add interactivity to the a graph that was created with ipycytoscape. This article/notebook follows the three articles published before and is the continuation in part of those.

### Our role as coder for the exercise
We have to (further) create a GUI representing the European Railway system.
Up to the previous article/notebook no interaction was still added to the graph.

### What do you need to know?
You should have read the previous three notebooks/articles in this series:   
- cyto_from_scratch_1
- cyto_from_scratch_2
- cyto_from_scratch_1_with_pandas
You should have some notions of pandas.
You should have some notions of ipywidgets and voila.

### Recap, where are we?
We know know how to plot or render graphs with ipycytoscape.
We know that you can use a function using pandas to generate the graph to be rendered.
we know how to change the basic layout of an ipycytoscape graph.

### How to learn/read this article.
My suggestion is to read the article and then download the code from the repo:

which contains the notebooks equivalent to every article.


## Pandas: The data.
As I pointed out in the previous articles/notebooks a function can be developed to create a graph based on dataframes representing nodes and edges of a graph, i.e. in our example nodes being the rail stations and edges being the connection between rail stations.

Following also the previously approach I would like every article to be self content, so without delay lets import the neccesary libraries and the mentioned function.

In [1]:
import ipycytoscape
import json
import ipywidgets
import pandas as pd


def transform_into_ipycytoscape(nodes_df,edges_df):
    
    nodes_df.fillna('null',inplace=True)
    edges_df.fillna('null',inplace=True)
    
    nodes_dict = nodes_df.to_dict('records')
    edges_dict = edges_df.to_dict('records')
    
    # building nodes

    data_keys = ['id','label','classes']
    position_keys = ['position_x','position_y']
    rest_keys = ['score','idInt','name','score','group','removed','selected','selectable','locked','grabbed'
                 'grabbable']
    
    nodes_graph_list=[]
    for node in nodes_dict:
        dict_node = {}
        data_sub_dict = {'data':{el:node[el] for el in data_keys}}
        rest_sub_dict = {el:node[el] for el in node.keys() if el in rest_keys}
        posi_sub_dict = {}
        
        if 'position_x' in node.keys() and 'position_y' in node.keys():
            coordinates = dict()
            if node['position_x'] != 'null':
                coordinates['x'] = int(node['position_x'])
            if node['position_y'] != 'null':
                coordinates['y'] = int(node['position_y'])
            if coordinates != {}:
                posi_sub_dict = {'position':coordinates}
        
        dict_node = {**data_sub_dict,**rest_sub_dict,**posi_sub_dict}
        nodes_graph_list.append(dict_node)
        
    
    # building edges
    
    data_keys  = ['id','source','target']
    data_keys2 = ['label','classes']
    rest_keys  = ['score','weight','group','networkId','networkGroupId','intn','rIntnId','group','removed','selected','selectable','locked','grabbed','grabbable','classes']
    position_keys = ['position_x','position_y']
    
    edges_graph_list = []
    for edge in edges_dict:
        dict_edge = {}
        data_sub_dict = {el:edge[el] for el in data_keys}
        data_sub_dict2 = {el:edge[el] for el in edge.keys() if el in data_keys2}
        rest_sub_dict = {el:edge[el] for el in edge.keys() if el in rest_keys}
        
        dict_edge = {'data':{**data_sub_dict,**data_sub_dict},**rest_sub_dict}
        edges_graph_list.append(dict_edge)
    
    total_graph_dict = {'nodes': nodes_graph_list, 'edges':edges_graph_list}
    
    # building the style
    all_node_style = ['background-color','background-opacity',
                     'font-family','font-size','label','width',
                     'shape','height','width','text-valign','text-halign']
    all_edge_style = ['background-color','background-opacity',
                     'font-family','font-size','label','width','line-color',
                     ]
    
    total_style_dict = {}
    style_elements=[]
    for node in nodes_dict:
        node_dict = {'selector': f'node[id = \"{node["id"]}\"]'}
        style_dict ={"style": { el:node[el] for el in node.keys() if el in all_node_style}}
        node_dict.update(style_dict)
        style_elements.append(node_dict)
    
    for edge in edges_dict:
        edge_dict = {'selector': f'edge[id = \"{edge["id"]}\"]'}
        style_dict ={"style": { el:edge[el] for el in edge.keys() if el in all_edge_style}}
        edge_dict.update(style_dict)
        style_elements.append(edge_dict)
    
    # the graph
    data_graph = json.dumps(total_graph_dict)
    json_to_python = json.loads(data_graph)
    result_cyto = ipycytoscape.CytoscapeWidget()
    result_cyto.graph.add_graph_from_json(json_to_python)    
    result_cyto.set_style(style_elements)    
    
    return result_cyto

### The data: The Rail stations -> Nodes of the graph
(see previous articles for understanding the follwoing JSON data and the pandas data manipulations)

In [2]:
stations = [{'id': 'BER','country': 'Germany','classes': 'east','label': 'BER Hbf','passengers': 400000,'position_x':0,'position_y':0},
        {'id': 'MUN','country': 'Germany','classes': 'west','label': 'MUN Hbf','passengers': 200000},
        {'id': 'FRA','country': 'Germany','classes': 'west','label': 'HBf FRA','passengers': 200000},
        {'id': 'HAM','country': 'Germany', 'classes': 'west','label': 'HBf HAM','passengers': 150000},
        {'id': 'LEP','country': 'Germany','label': 'HBf LEP','classes': 'east','passengers': 50000},
        {'id': 'NUR','country': 'Germany','label': 'HBf NUR','classes': 'west','passengers': 50000},
        {'id': 'PAR', 'country': 'France','label': 'PAR CS', 'classes': '', 'passengers': 350000},
        {'id': 'MIL', 'country': 'Italy', 'label': 'MIL CS','classes': '', 'passengers': 250000},
        {'id': 'BAR', 'country': 'Spain', 'label': 'BAR CS','classes': '', 'passengers': 200000},
        {'id': 'LYO', 'country': 'France','label': 'LYO CS','classes': '', 'passengers': 200000},
        {'id': 'LON', 'country': 'UK','label': 'LON CS','classes': '', 'passengers': 400000},
        {'id': 'LIS', 'country': 'Portugal','label': 'LIS CS','classes': '', 'passengers': 200000,'position_x':1000,'position_y':10000},
        {'id': 'WAR', 'country': 'Poland','label': 'WAR CS','classes': '', 'passengers': 200000},
           ]
stations_df = pd.DataFrame(stations)
stations_df.loc[stations_df['country'] != 'Germany','classes'] = 'EU'
stations_df['background-color']=''
stations_df.loc[stations_df['country'] != 'Germany','background-color'] = 'blue'
stations_df.loc[stations_df['country'] == 'Germany','background-color'] = 'orange'
stations_df.loc[stations_df['id'] == 'BER','background-color']  = 'yellow'
stations_df

,id,country,classes,label,passengers,position_x,position_y,background-color
0,BER,Germany,east,BER Hbf,400000,0.0,0.0,yellow
1,MUN,Germany,west,MUN Hbf,200000,NaN,NaN,orange
2,FRA,Germany,west,HBf FRA,200000,NaN,NaN,orange
3,HAM,Germany,west,HBf HAM,150000,NaN,NaN,orange
4,LEP,Germany,east,HBf LEP,50000,NaN,NaN,orange
5,NUR,Germany,west,HBf NUR,50000,NaN,NaN,orange
6,PAR,France,EU,PAR CS,350000,NaN,NaN,blue
7,MIL,Italy,EU,MIL CS,250000,NaN,NaN,blue
8,BAR,Spain,EU,BAR CS,200000,NaN,NaN,blue
9,LYO,France,EU,LYO CS,200000,NaN,NaN,blue


### The data: The Rail connections -> Edges of the graph
(see previous articles for understanding the follwoing JSON data and the pandas data manipulations)

In [3]:
rail_lines = [{'id': 'line1', 'source': 'BER', 'target': 'MUN', 'speed': '200km/h'},
              {'id': 'line2', 'source': 'MUN', 'target': 'FRA', 'speed': '200km/h'},
              {'id': 'line3', 'source': 'FRA', 'target': 'BER', 'speed': '250km/h'}, 
              {'id': 'line4', 'source': 'BER', 'target': 'HAM', 'speed': '300km/h'}, 
              {'id': 'line5', 'source': 'BER', 'target': 'LEP', 'speed': '300km/h'},
              {'id': 'line6', 'source': 'NUR', 'target': 'LEP', 'speed': '150km/h'}, 
              {'id': 'line7', 'source': 'NUR', 'target': 'FRA', 'speed': '150km/h'},
              {'id': 'line8', 'source': 'BER', 'target': 'PAR', 'speed': '400km/h'}, 
              {'id': 'line9', 'source': 'PAR', 'target': 'LYO', 'speed': '400km/h'}, 
              {'id': 'line10', 'source': 'LYO', 'target': 'BAR', 'speed': '400km/h'},
              {'id': 'line11', 'source': 'PAR', 'target': 'LON', 'speed': '400km/h'},
              {'id': 'line12', 'source': 'BER', 'target': 'WAR', 'speed': '400km/h'},
              {'id': 'line13', 'source': 'BAR', 'target': 'LIS', 'speed': '400km/h'},
              {'id': 'line14', 'source': 'LYO', 'target': 'MIL', 'speed': '200km/h'},
              {'id': 'line15', 'source': 'BAR', 'target': 'MIL', 'speed': '200km/h'},
              {'id': 'line16', 'source': 'PAR', 'target': 'BAR', 'speed': '400km/h'},
              {'id': 'line17', 'source': 'MUN', 'target': 'PAR', 'speed': '200km/h'},
              {'id': 'line18', 'source': 'LYO', 'target': 'LON', 'speed': '200km/h'}
             ]
rails_df = pd.DataFrame(rail_lines,columns=['id','source','target','speed'])
rails_df['label'] = rails_df['speed']
rails_df['background-color'] = 'black'
rails_df

,id,source,target,speed,label,background-color
0,line1,BER,MUN,200km/h,200km/h,black
1,line2,MUN,FRA,200km/h,200km/h,black
2,line3,FRA,BER,250km/h,250km/h,black
3,line4,BER,HAM,300km/h,300km/h,black
4,line5,BER,LEP,300km/h,300km/h,black
5,line6,NUR,LEP,150km/h,150km/h,black
6,line7,NUR,FRA,150km/h,150km/h,black
7,line8,BER,PAR,400km/h,400km/h,black
8,line9,PAR,LYO,400km/h,400km/h,black
9,line10,LYO,BAR,400km/h,400km/h,black


In [4]:
G=transform_into_ipycytoscape(stations_df,rails_df)

In [5]:
G.get_layout()

{'name': 'cola'}

G.set_layout(name = 'cola',
                      nodeSpacing = 50,
                      edgeLengthVal = 45,
                      animate = True,
                      randomize = False,
                      maxSimulationTime = 1500)

In [5]:
G.layout.height = '1000px'
G.graph.nodes

[Node(data={'id': 'BER', 'label': 'BER Hbf', 'classes': 'east'}, position={'x': 0, 'y': 0}),
 Node(data={'id': 'MUN', 'label': 'MUN Hbf', 'classes': 'west'}, position={}),
 Node(data={'id': 'FRA', 'label': 'HBf FRA', 'classes': 'west'}, position={}),
 Node(data={'id': 'HAM', 'label': 'HBf HAM', 'classes': 'west'}, position={}),
 Node(data={'id': 'LEP', 'label': 'HBf LEP', 'classes': 'east'}, position={}),
 Node(data={'id': 'NUR', 'label': 'HBf NUR', 'classes': 'west'}, position={}),
 Node(data={'id': 'PAR', 'label': 'PAR CS', 'classes': 'EU'}, position={}),
 Node(data={'id': 'MIL', 'label': 'MIL CS', 'classes': 'EU'}, position={}),
 Node(data={'id': 'BAR', 'label': 'BAR CS', 'classes': 'EU'}, position={}),
 Node(data={'id': 'LYO', 'label': 'LYO CS', 'classes': 'EU'}, position={}),
 Node(data={'id': 'LON', 'label': 'LON CS', 'classes': 'EU'}, position={}),
 Node(data={'id': 'LIS', 'label': 'LIS CS', 'classes': 'EU'}, position={'x': 1000, 'y': 10000}),
 Node(data={'id': 'WAR', 'label': '

-----------------------------

I would like you to imagine is that the graph that is going to be built in this notebook is real part of an interface. The interface is supposed to be used by people sit behind the computer and looking and manipulating the graph. Something similar to an air traffic controller. So the colourings and appearance of the graph might be changing continuously depending on the number of passengers on trains, number of trains in stations, different colours for different train speeds etc.   
So if were the coder of the project you will have to reflect all that data into the GUI.

The most natural way to work with tabular data in data science with python is pandas. 

Note 1: The graph and interface can be deploy with voilà (it is not the purpose of this article to deepen into voilà, you just need to know that there are tools to render, i.e. display, ipycytoscape graphs into a browser)

Note 2: Ipycytoscape includes already an API that allows to pass a pandas DataFrame to a graph constructor (see here: https://ipycytoscape.readthedocs.io/en/latest/examples/pandas.html). Nevertheless, my approach is slightly different; I would like to show a way to manipulate and change graphs using strictly pandas.

Imagine you are given the data in a table called "stations.xls" and another table called "railconnections.xls".

Note: I pasted here a dictionary in order for you to be able to follow along without the real need of reading an external file and for article self-content purposes)

In [6]:
import ipycytoscape
import json
import ipywidgets
import pandas as pd

stations = [{'id': 'BER','country': 'Germany','classes': 'east','label': 'BER Hbf','passengers': 400000},
        {'id': 'MUN','country': 'Germany','classes': 'west','label': 'MUN Hbf','passengers': 200000},
        {'id': 'FRA','country': 'Germany','classes': 'west','label': 'HBf FRA','passengers': 200000},
        {'id': 'HAM','country': 'Germany', 'classes': 'west','label': 'HBf HAM','passengers': 150000},
        {'id': 'LEP','country': 'Germany','label': 'HBf LEP','classes': 'east','passengers': 50000},
        {'id': 'NUR','country': 'Germany','label': 'HBf NUR','classes': 'west','passengers': 50000},
        {'id': 'PAR', 'country': 'France','label': 'PAR CS', 'classes': '', 'passengers': 350000},
        {'id': 'MIL', 'country': 'Italy', 'label': 'MIL CS','classes': '', 'passengers': 250000},
        {'id': 'BAR', 'country': 'Spain', 'label': 'BAR CS','classes': '', 'passengers': 200000},
        {'id': 'LYO', 'country': 'France','label': 'LYO CS','classes': '', 'passengers': 200000}]

stations_df = pd.DataFrame(stations)
stations_df

,id,country,classes,label,passengers
0,BER,Germany,east,BER Hbf,400000
1,MUN,Germany,west,MUN Hbf,200000
2,FRA,Germany,west,HBf FRA,200000
3,HAM,Germany,west,HBf HAM,150000
4,LEP,Germany,east,HBf LEP,50000
5,NUR,Germany,west,HBf NUR,50000
6,PAR,France,,PAR CS,350000
7,MIL,Italy,,MIL CS,250000
8,BAR,Spain,,BAR CS,200000
9,LYO,France,,LYO CS,200000


And here the file "railconnections.xls".

In [7]:
rail_lines = [{'id': 'line1', 'source': 'BER', 'target': 'MUN', 'speed': '200km/h'},
         {'id': 'line2', 'source': 'MUN', 'target': 'FRA', 'speed': '200km/h'},
         {'id': 'line3', 'source': 'FRA', 'target': 'BER', 'speed': '250km/h'}, 
         {'id': 'line4', 'source': 'BER', 'target': 'HAM', 'speed': '300km/h'}, 
         {'id': 'line5', 'source': 'BER', 'target': 'LEP', 'speed': '300km/h'},
         {'id': 'line6', 'source': 'NUR', 'target': 'LEP', 'speed': '150km/h'}, 
         {'id': 'line7', 'source': 'NUR', 'target': 'FRA', 'speed': '150km/h'},
         {'id': 'line8', 'source': 'BER', 'target': 'PAR', 'speed': '400km/h'}, 
         {'id': 'line9', 'source': 'PAR', 'target': 'LYO', 'speed': '400km/h'}, 
         {'id': 'line10', 'source': 'LYO', 'target': 'BAR', 'speed': '400km/h'},
         {'id': 'line11', 'source': 'LYO', 'target': 'MIL', 'speed': '200km/h'}]
rails_df = pd.DataFrame(rail_lines,columns=['id','source','target','speed'])
rails_df['label'] = rails_df['speed']
rails_df['background-color'] = 'black'
rails_df

,id,source,target,speed,label,background-color
0,line1,BER,MUN,200km/h,200km/h,black
1,line2,MUN,FRA,200km/h,200km/h,black
2,line3,FRA,BER,250km/h,250km/h,black
3,line4,BER,HAM,300km/h,300km/h,black
4,line5,BER,LEP,300km/h,300km/h,black
5,line6,NUR,LEP,150km/h,150km/h,black
6,line7,NUR,FRA,150km/h,150km/h,black
7,line8,BER,PAR,400km/h,400km/h,black
8,line9,PAR,LYO,400km/h,400km/h,black
9,line10,LYO,BAR,400km/h,400km/h,black


Lets now add the neccesary data to the table. The class of the new stations is 'EU' (look part one and two of this series for understanding classes).    
Every EU rail station should be displayed orange, and the rest of the stations blue, except for the German capital which will be yellow.    
Lets add this data to the DataFrame.

In [8]:
stations_df.loc[stations_df['country'] != 'Germany','classes'] = 'EU'
stations_df['background-color']=''
stations_df.loc[stations_df['country'] == 'Germany','background-color'] = 'blue'
stations_df.loc[stations_df['country'] != 'Germany','background-color'] = 'orange'
stations_df.loc[stations_df['id'] == 'BER','background-color']  = 'yellow'
stations_df

,id,country,classes,label,passengers,background-color
0,BER,Germany,east,BER Hbf,400000,yellow
1,MUN,Germany,west,MUN Hbf,200000,blue
2,FRA,Germany,west,HBf FRA,200000,blue
3,HAM,Germany,west,HBf HAM,150000,blue
4,LEP,Germany,east,HBf LEP,50000,blue
5,NUR,Germany,west,HBf NUR,50000,blue
6,PAR,France,EU,PAR CS,350000,orange
7,MIL,Italy,EU,MIL CS,250000,orange
8,BAR,Spain,EU,BAR CS,200000,orange
9,LYO,France,EU,LYO CS,200000,orange


The above showed data tables contain the attributes of the stations and rails connections.
A method to pass all this data to ipycytoscape is needed.   

The following method manipulates both tables above in order to output a JSON file that can then be passed to ipycytoscape. The method itself returns the ipycytoscape graph.   
Afterwards it is only necessary to plot it.

In [9]:
def transform_into_ipycytoscape(nodes_df,edges_df):
    
    if 'classes' not in nodes_df.columns:
        nodes_df['classes'] = 'no_class'
        
    if 'classes' not in edges_df.columns:
        edges_df['classes'] = 'no_class'
        
    nodes_dict = nodes_df.to_dict('records')
    edges_dict = edges_df.to_dict('records')
    
    
        
    
    # building nodes

    data_keys = ['id','label','classes']
    position_keys = ['position_x','position_y']
    rest_keys = ['score','idInt','name','score','group','removed','selected','selectable','locked','grabbed'
                 'grabbable']
    
    nodes_graph_list=[]
    for node in nodes_dict:
        dict_node = {}
        data_sub_dict = {'data':{el:node[el] for el in data_keys}}
        rest_sub_dict = {el:node[el] for el in node.keys() if el in rest_keys}
        posi_sub_dict = {}
        if 'position_x' in node.keys() and 'position_y' in node.keys():
            #print({el:node[el] for el in node.keys() if el in position_keys})
            posi_sub_dict = {'position':{'x':node['position_x'],'y':node['position_y']}}
                                         
        dict_node = {**data_sub_dict,**rest_sub_dict,**posi_sub_dict}
        nodes_graph_list.append(dict_node)
        
    
    # building edges
    
    data_keys  = ['id','source','target']
    data_keys2 = ['label','classes']
    rest_keys  = ['score','weight','group','networkId','networkGroupId','intn','rIntnId','group','removed','selected','selectable','locked','grabbed','grabbable','classes']
    position_keys = ['position_x','position_y']
    
    edges_graph_list = []
    for edge in edges_dict:
        dict_edge = {}
        data_sub_dict = {el:edge[el] for el in data_keys}
        data_sub_dict2 = {el:edge[el] for el in edge.keys() if el in data_keys2}
        rest_sub_dict = {el:edge[el] for el in edge.keys() if el in rest_keys}
        
        dict_edge = {'data':{**data_sub_dict,**data_sub_dict},**rest_sub_dict}
        edges_graph_list.append(dict_edge)
    
    total_graph_dict = {'nodes': nodes_graph_list, 'edges':edges_graph_list}
    
    # building the style
    all_node_style = ['background-color','background-opacity',
                     'font-family','font-size','label','width',
                     'shape','height','width','text-valign','text-halign']
    all_edge_style = ['background-color','background-opacity',
                     'font-family','font-size','label','width','line-color',
                     ]
    
    total_style_dict = {}
    style_elements=[]
    for node in nodes_dict:
        node_dict = {'selector': f'node[id = \"{node["id"]}\"]'}
        style_dict ={"style": { el:node[el] for el in node.keys() if el in all_node_style}}
        node_dict.update(style_dict)
        style_elements.append(node_dict)
    
    for edge in edges_dict:
        edge_dict = {'selector': f'edge[id = \"{edge["id"]}\"]'}
        style_dict ={"style": { el:edge[el] for el in edge.keys() if el in all_edge_style}}
        edge_dict.update(style_dict)
        style_elements.append(edge_dict)
    
    # the graph
    data_graph = json.dumps(total_graph_dict)
    json_to_python = json.loads(data_graph)
    result_cyto = ipycytoscape.CytoscapeWidget()
    result_cyto.graph.add_graph_from_json(json_to_python)    
    result_cyto.set_style(style_elements)    
    
    return result_cyto

G=transform_into_ipycytoscape(stations_df,rails_df)
display(G)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "BER"]', 'style': …

But I started this article saying that I would show why being able to work directly with tables is a good way to go.   
Now assume that you want to change the colours of the Graph in the following way.  
Stations with more than 200000 passengers should be rendered red and the rest green.  
The high-speed rail lines have should be painted red. ( high-speed line is considered the one with more or equal than 300km/h)
We can do that operating over the data frame and pass the resulting data frame to the ipycytoscape constructor of the method above defined.
What are then the CSS affected attributes? For the nodes is the 'background-color' (we already used it) and for the lines is the 'line-color'.
Let's see.

In [10]:
stations_df['background-color'] = stations_df['passengers'].apply(lambda x: 'red' if x>200000 else 'blue')
rails_df['line-color'] = rails_df['label'].apply(lambda x: 'red' if x in ['400km/h','300km/h'] else 'green')

With only two lines we added the neccesary changes and the dtaframes are now as follows.

In [11]:
stations_df

,id,country,classes,label,passengers,background-color
0,BER,Germany,east,BER Hbf,400000,red
1,MUN,Germany,west,MUN Hbf,200000,blue
2,FRA,Germany,west,HBf FRA,200000,blue
3,HAM,Germany,west,HBf HAM,150000,blue
4,LEP,Germany,east,HBf LEP,50000,blue
5,NUR,Germany,west,HBf NUR,50000,blue
6,PAR,France,EU,PAR CS,350000,red
7,MIL,Italy,EU,MIL CS,250000,red
8,BAR,Spain,EU,BAR CS,200000,blue
9,LYO,France,EU,LYO CS,200000,blue


In [12]:
rails_df

,id,source,target,speed,label,background-color,classes,line-color
0,line1,BER,MUN,200km/h,200km/h,black,no_class,green
1,line2,MUN,FRA,200km/h,200km/h,black,no_class,green
2,line3,FRA,BER,250km/h,250km/h,black,no_class,green
3,line4,BER,HAM,300km/h,300km/h,black,no_class,red
4,line5,BER,LEP,300km/h,300km/h,black,no_class,red
5,line6,NUR,LEP,150km/h,150km/h,black,no_class,green
6,line7,NUR,FRA,150km/h,150km/h,black,no_class,green
7,line8,BER,PAR,400km/h,400km/h,black,no_class,red
8,line9,PAR,LYO,400km/h,400km/h,black,no_class,red
9,line10,LYO,BAR,400km/h,400km/h,black,no_class,red


In [13]:
G=transform_into_ipycytoscape(stations_df,rails_df)
display(G)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "BER"]', 'style': …

### Changing appearance of the edges & nodes with the help of pandas
In order to further illustrate the power of this approach I will add another layout change.
A new security regulation was passed in parliament and all the stations where a high train speed arrives should have special anti-fire measures. So the GUI should show in violet all the stations where at least a high speed train arrives and in green those in which that is not the case.   
Stations in which at least one high speed line arrives 

In [14]:
rails_df['high-speed'] = rails_df.apply(lambda x: [x.target,x.source] if x.speed in ['400km/h'] else [], axis=1)

In [15]:
rails_df

,id,source,target,speed,label,background-color,classes,line-color,high-speed
0,line1,BER,MUN,200km/h,200km/h,black,no_class,green,[]
1,line2,MUN,FRA,200km/h,200km/h,black,no_class,green,[]
2,line3,FRA,BER,250km/h,250km/h,black,no_class,green,[]
3,line4,BER,HAM,300km/h,300km/h,black,no_class,red,[]
4,line5,BER,LEP,300km/h,300km/h,black,no_class,red,[]
5,line6,NUR,LEP,150km/h,150km/h,black,no_class,green,[]
6,line7,NUR,FRA,150km/h,150km/h,black,no_class,green,[]
7,line8,BER,PAR,400km/h,400km/h,black,no_class,red,"[PAR, BER]"
8,line9,PAR,LYO,400km/h,400km/h,black,no_class,red,"[LYO, PAR]"
9,line10,LYO,BAR,400km/h,400km/h,black,no_class,red,"[BAR, LYO]"


In [16]:
stations_high_speed_arriving = rails_df['high-speed'].to_list()
list_of_HS_stations =list(set([item for sublist in stations_high_speed_arriving for item in sublist]))
list_of_HS_stations

['BER', 'BAR', 'LYO', 'PAR']

### changing color of nodes
Those are the stations from which at least one high speed train departs.   
Lets change the color of the stations in the stations dataframe.


In [17]:
stations_df['background-color'] = stations_df['id'].apply(lambda x: 'violet' if x in list_of_HS_stations else 'yellow')

In [18]:
G=transform_into_ipycytoscape(stations_df,rails_df)
display(G)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "BER"]', 'style': …

### changing shapes of nodes
Next the stations with more than 350000 passengers are going to be plotted as square to differenciate them from the other ones that are smaller.

In [19]:
stations_df['shape'] = stations_df['passengers'].apply(lambda x: 'rectangle' if x>=350000 else '') 

In [20]:
stations_df

,id,country,classes,label,passengers,background-color,shape
0,BER,Germany,east,BER Hbf,400000,violet,rectangle
1,MUN,Germany,west,MUN Hbf,200000,yellow,
2,FRA,Germany,west,HBf FRA,200000,yellow,
3,HAM,Germany,west,HBf HAM,150000,yellow,
4,LEP,Germany,east,HBf LEP,50000,yellow,
5,NUR,Germany,west,HBf NUR,50000,yellow,
6,PAR,France,EU,PAR CS,350000,violet,rectangle
7,MIL,Italy,EU,MIL CS,250000,yellow,
8,BAR,Spain,EU,BAR CS,200000,violet,
9,LYO,France,EU,LYO CS,200000,violet,


In [21]:
G=transform_into_ipycytoscape(stations_df,rails_df)
display(G)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "BER"]', 'style': …

### changing the size of the nodes

We want to make the size of the nodes proportional to the amount of passengers of every station.    
In this way the graph will reflect more appropiately the (relative) weight of every station in the whole net.   
If you google for normalizing data you will find that often time the preprocessing library of sklearn is used. In order to not to make things here more complicated a formula will be used without making use of yet another very heavy lybrary.

formula = value / (max_value - min_value)   

Again we can create a column in our nodes dataframe representing the score of every of station.

In [22]:
stations_df['score'] = stations_df['passengers'].apply(lambda x: (x-stations_df['passengers'].min())/(stations_df['passengers'].max()-stations_df['passengers'].min()))

In [23]:
stations_df

,id,country,classes,label,passengers,background-color,shape,score
0,BER,Germany,east,BER Hbf,400000,violet,rectangle,1.000000
1,MUN,Germany,west,MUN Hbf,200000,yellow,,0.428571
2,FRA,Germany,west,HBf FRA,200000,yellow,,0.428571
3,HAM,Germany,west,HBf HAM,150000,yellow,,0.285714
4,LEP,Germany,east,HBf LEP,50000,yellow,,0.000000
5,NUR,Germany,west,HBf NUR,50000,yellow,,0.000000
6,PAR,France,EU,PAR CS,350000,violet,rectangle,0.857143
7,MIL,Italy,EU,MIL CS,250000,yellow,,0.571429
8,BAR,Spain,EU,BAR CS,200000,violet,,0.428571
9,LYO,France,EU,LYO CS,200000,violet,,0.428571


We have a normalized column with the size of the rail stations. if we assume a size of 20px minimun we have to add 20 to the column.

In [24]:
stations_df.drop('shape', axis =1, inplace = True)

In [25]:
stations_df['width'] = stations_df['score'] * 40 + 40
stations_df['height'] = stations_df['score'] * 40 + 40

In [26]:
stations_df

,id,country,classes,label,passengers,background-color,score,width,height
0,BER,Germany,east,BER Hbf,400000,violet,1.000000,80.000000,80.000000
1,MUN,Germany,west,MUN Hbf,200000,yellow,0.428571,57.142857,57.142857
2,FRA,Germany,west,HBf FRA,200000,yellow,0.428571,57.142857,57.142857
3,HAM,Germany,west,HBf HAM,150000,yellow,0.285714,51.428571,51.428571
4,LEP,Germany,east,HBf LEP,50000,yellow,0.000000,40.000000,40.000000
5,NUR,Germany,west,HBf NUR,50000,yellow,0.000000,40.000000,40.000000
6,PAR,France,EU,PAR CS,350000,violet,0.857143,74.285714,74.285714
7,MIL,Italy,EU,MIL CS,250000,yellow,0.571429,62.857143,62.857143
8,BAR,Spain,EU,BAR CS,200000,violet,0.428571,57.142857,57.142857
9,LYO,France,EU,LYO CS,200000,violet,0.428571,57.142857,57.142857


In [27]:
G=transform_into_ipycytoscape(stations_df,rails_df)
display(G)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "BER"]', 'style': …

The same can be done with the rail connections. The higher the speed the thicker we want to pain the rail connections.


In [28]:
rails_df['speed_int']=rails_df['speed'].str.replace('km/h','').astype('int')
min_speed = rails_df['speed_int'].min()
max_speed = rails_df['speed_int'].max()
span_speed = max_speed-min_speed

rails_df['score'] = rails_df['speed_int'].apply(lambda x: (x-min_speed)/span_speed)

In [29]:
rails_df['width'] = rails_df['score'] *7 +5


In [30]:
G=transform_into_ipycytoscape(stations_df,rails_df)
display(G)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "BER"]', 'style': …

-------------

In [31]:
stations2_df = stations_df.copy(deep=True)

In [32]:
stations2_df.loc[0, 'position_x'] = 0
stations2_df.loc[0, 'position_y'] = 0
for i in range(1,len(stations2_df)):
    stations2_df.loc[i, 'position_x'] = i*10
    stations2_df.loc[i, 'position_y'] = i*5
    
stations2_df.drop(['score','width','height'], axis=1, inplace=True)
stations2_df['label'] = stations2_df['position_x'].astype(str) + '-' + stations2_df['position_y'].astype(str)

In [33]:
rails2_df= rails_df.copy(deep=True)
rails2_df.drop(['label','speed','score','width'],axis=1,inplace=True)
rails2_df

,id,source,target,background-color,classes,line-color,high-speed,speed_int
0,line1,BER,MUN,black,no_class,green,[],200
1,line2,MUN,FRA,black,no_class,green,[],200
2,line3,FRA,BER,black,no_class,green,[],250
3,line4,BER,HAM,black,no_class,red,[],300
4,line5,BER,LEP,black,no_class,red,[],300
5,line6,NUR,LEP,black,no_class,green,[],150
6,line7,NUR,FRA,black,no_class,green,[],150
7,line8,BER,PAR,black,no_class,red,"[PAR, BER]",400
8,line9,PAR,LYO,black,no_class,red,"[LYO, PAR]",400
9,line10,LYO,BAR,black,no_class,red,"[BAR, LYO]",400


In [34]:
print(ipycytoscape.__version__)
G=transform_into_ipycytoscape(stations2_df,rails2_df)
G.graph.nodes

1.0.4


[Node(data={'id': 'BER', 'label': '0.0-0.0', 'classes': 'east'}, position={'x': 0.0, 'y': 0.0}),
 Node(data={'id': 'MUN', 'label': '10.0-5.0', 'classes': 'west'}, position={'x': 10.0, 'y': 5.0}),
 Node(data={'id': 'FRA', 'label': '20.0-10.0', 'classes': 'west'}, position={'x': 20.0, 'y': 10.0}),
 Node(data={'id': 'HAM', 'label': '30.0-15.0', 'classes': 'west'}, position={'x': 30.0, 'y': 15.0}),
 Node(data={'id': 'LEP', 'label': '40.0-20.0', 'classes': 'east'}, position={'x': 40.0, 'y': 20.0}),
 Node(data={'id': 'NUR', 'label': '50.0-25.0', 'classes': 'west'}, position={'x': 50.0, 'y': 25.0}),
 Node(data={'id': 'PAR', 'label': '60.0-30.0', 'classes': 'EU'}, position={'x': 60.0, 'y': 30.0}),
 Node(data={'id': 'MIL', 'label': '70.0-35.0', 'classes': 'EU'}, position={'x': 70.0, 'y': 35.0}),
 Node(data={'id': 'BAR', 'label': '80.0-40.0', 'classes': 'EU'}, position={'x': 80.0, 'y': 40.0}),
 Node(data={'id': 'LYO', 'label': '90.0-45.0', 'classes': 'EU'}, position={'x': 90.0, 'y': 45.0})]

In [35]:
display(G)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "BER"]', 'style': …

In [6]:
nodes4 = [{'id': 'UNO', 'label': '0.0-0.0','position_x': 0.0, 'position_y': 0.0},
{'id': 'DOS', 'label': '10.0-5.0', 'position_x': 10.0, 'position_y': 5.0},
{'id': 'TRES', 'label': '20.0-10.0', 'position_x': 20.0, 'position_y': 10.0},
{'id': 'CUATRO', 'label': '30.0-15.0', 'position_x': 30.0, 'position_y': 15.0},
{'id': 'CINCO', 'label': '30.0-15.0', 'position_x': 30.0, 'position_y': 15.0}]

edges4 = [{'id': 'line1', 'source': 'UNO', 'target': 'DOS', 'label': 'line1'},
          {'id': 'line2', 'source': 'UNO', 'target': 'TRES', 'label':'line2'},
          {'id': 'line3', 'source': 'DOS', 'target': 'TRES', 'label': 'line3'},
          {'id': 'line4', 'source': 'TRES', 'target': 'CUATRO', 'label': 'line4'}, 
          {'id': 'line5', 'source': 'CUATRO', 'target': 'CINCO', 'label': 'line5'}
             ]

nodes4_df = pd.DataFrame(nodes4)
edges4_df = pd.DataFrame(edges4)

G=transform_into_ipycytoscape(nodes4_df,edges4_df)

KeyError: 'classes'

In [36]:
G.graph.nodes

G.graph.edges

G.cytoscape_style


G.get_style()

G

G.get_layout()